 # PROYECTO SPRINT 9

### OBJETIVO DEL PROYECTO
<p>
En Beta Bank, los clientes están abandonando gradualmente el banco cada mes. Este fenómeno es costoso, ya que los banqueros han descubierto que es más económico retener a los clientes existentes que adquirir nuevos. El objetivo principal de este proyecto es predecir si un cliente abandonará el banco pronto, utilizando datos históricos sobre el comportamiento de los clientes y la terminación de contratos con el banco.</P>

<P>Resultados Esperados:</P>
<P>
Se espera desarrollar un modelo predictivo con un alto valor F1-score, que indique su capacidad para predecir con precisión el abandono de clientes. Se establece un objetivo mínimo de un valor F1 de 0.59.</P>

### 1. Descarga y prepara los datos:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score


In [2]:
#Cargar archivo DataFrame
df = pd.read_csv('Churn.csv')

In [3]:
display(df)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
#Validamos información estadistica del DataFrame
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
#Remplazamos valores ausentes con la media y evitamos perder información del df
mean_tenure = df['Tenure'].mean()
df['Tenure'].fillna(mean_tenure,inplace=True)

#Verificamos si se completaron los valores ausentes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


<P>Remplazamos valores ausentes con la media para no perder ninguna fila y poder continuar el análisis con la mayor cantidad de datos y el análisis siga siendo representativo. </P>

### 2. Examinar el equilibrio de clases:

In [7]:
#Identificar la distribución de la variable objetivito 'Exited'
class_distribution = df['Exited'].value_counts()
print(class_distribution)

Exited
0    7963
1    2037
Name: count, dtype: int64


### 3. Entrenar el modelo sin tener en cuenta el desequilibrio

In [8]:
# Convertir variables categóricas en variables dummy
features_encoded = pd.get_dummies(df, columns=['Geography', 'Gender'])

# Dividir los datos en características y variable objetivo
features = features_encoded.drop(columns=['Exited','RowNumber','CustomerId','Surname'])
target = features_encoded['Exited']

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features, target, test_size=0.25, random_state=12345)

# Crear y entrenar el modelo de árbol de decisión
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train1, target_train1)
predicted_valid = model.predict(features_valid1)

print(predicted_valid)

[0 0 0 ... 0 1 1]


Se utilizó la codificacióne One-hot (dummy) porque tenía características categóricas en mis datos y no se puede manejar automáticamente. 
Para resolver este problema, se reprocesó los datos para manejar las características categóricas y lograr convertirlas en carácterísticas numéricas. 

### 4. Mejora la calidad del modelo:

In [10]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
df_train_valid, df_test = train_test_split(features_encoded, test_size=0.2, random_state=12345)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345)

# Extraer características y objetivos para entrenamiento
features_train = df_train.drop(columns=['Exited','RowNumber','CustomerId','Surname'])
target_train = df_train['Exited']

# Extraer características y objetivos para validación
features_valid = df_valid.drop(columns=['Exited','RowNumber','CustomerId','Surname'])
target_valid = df_valid['Exited']

# Extraer características y objetivos para prueba
features_test = df_test.drop(columns=['Exited','RowNumber','CustomerId','Surname'])
target_test = df_test['Exited']

# Combinar características y variable objetivo para el conjunto de entrenamiento
train_data = pd.concat([features_train, target_train], axis=1)

# Separar muestras por clase
class_0 = train_data[train_data['Exited'] == 0]
class_1 = train_data[train_data['Exited'] == 1]

# Upsample de la clase minoritaria (clase 1)
class_1_upsampled = resample(class_1, replace=True, n_samples=len(class_0), random_state=12345)

# Downsample de la clase mayoritaria (clase 0)
class_0_downsampled = resample(class_0, replace=False, n_samples=len(class_1), random_state=12345)

# Combina las muestras de las dos clases luego de upsample y downsample
upsampled_train_data = pd.concat([class_0, class_1_upsampled])
downsampled_train_data = pd.concat([class_1, class_0_downsampled])

# Dividir características y variable objetivo después del remuestreo (upsampled)
features_train_upsampled = upsampled_train_data.drop(columns=['Exited'])
target_train_upsampled = upsampled_train_data['Exited']

# Dividir características y variable objetivo después del remuestreo (downsampled)
features_train_downsampled = downsampled_train_data.drop(columns=['Exited'])
target_train_downsampled = downsampled_train_data['Exited']

# Entrenar varios modelos con diferentes hiperparámetros (upsampled)
models_upsampled = []
for n_estimators in [50, 120, 180]:
    for max_depth in [None, 20, 40]:
        rf_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345)
        rf_classifier.fit(features_train_upsampled, target_train_upsampled)
        models_upsampled.append((rf_classifier, n_estimators, max_depth))

# Entrenar varios modelos con diferentes hiperparámetros (downsampled)
models_downsampled = []
for n_estimators in [50, 120, 180]:
    for max_depth in [None, 20, 40]:
        rf_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345)
        rf_classifier.fit(features_train_downsampled, target_train_downsampled)
        models_downsampled.append((rf_classifier, n_estimators, max_depth))

# Evaluar modelos en el conjunto de validación y seleccionar el mejor basado en F1-score y AUC-ROC (upsampled)
best_model_upsampled = None
best_f1_upsampled = 0
best_auc_roc_upsampled = 0
for model, n_estimators, max_depth in models_upsampled:
    target_pred_val = model.predict(features_valid)
    f1 = f1_score(target_valid, target_pred_val)
    auc_roc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])
    if f1 > best_f1_upsampled:
        best_model_upsampled = model
        best_f1_upsampled = f1
        best_auc_roc_upsampled = auc_roc
        best_params_upsampled = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Evaluar modelos en el conjunto de validación y seleccionar el mejor basado en F1-score y AUC-ROC (downsampled)
best_model_downsampled = None
best_f1_downsampled = 0
best_auc_roc_downsampled = 0
for model, n_estimators, max_depth in models_downsampled:
    target_pred_val = model.predict(features_valid)
    f1 = f1_score(target_valid, target_pred_val)
    auc_roc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])
    if f1 > best_f1_downsampled:
        best_model_downsampled = model
        best_f1_downsampled = f1
        best_auc_roc_downsampled = auc_roc
        best_params_downsampled = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Imprimir el reporte de clasificación del mejor modelo en el conjunto de validación (upsampled)
print("Reporte de clasificación en el conjunto de validación (upsampled):")
print(classification_report(target_valid, best_model_upsampled.predict(features_valid)))
print("Mejores parámetros (upsampled):", best_params_upsampled)
print("Puntaje F1 en el conjunto de validación (upsampled):", best_f1_upsampled)
print("AUC-ROC en el conjunto de validación (upsampled):", best_auc_roc_upsampled)

# Imprimir el reporte de clasificación del mejor modelo en el conjunto de validación (downsampled)
print("\nReporte de clasificación en el conjunto de validación (downsampled):")
print(classification_report(target_valid, best_model_downsampled.predict(features_valid)))
print("Mejores parámetros (downsampled):", best_params_downsampled)
print("Puntaje F1 en el conjunto de validación (downsampled):", best_f1_downsampled)
print("AUC-ROC en el conjunto de validación (downsampled):", best_auc_roc_downsampled)

# Evaluar el mejor modelo en el conjunto de prueba (upsampled)
target_pred_test_upsampled = best_model_upsampled.predict(features_test)

# Imprimir el reporte de clasificación del mejor modelo en el conjunto de prueba (upsampled)
print("\nReporte de clasificación en el conjunto de prueba (upsampled):")
print(classification_report(target_test, target_pred_test_upsampled))
print("Puntaje F1 en el conjunto de prueba (upsampled):", f1_score(target_test, target_pred_test_upsampled))
print("AUC-ROC en el conjunto de prueba (upsampled):", roc_auc_score(target_test, best_model_upsampled.predict_proba(features_test)[:, 1]))

# Evaluar el mejor modelo en el conjunto de prueba (downsampled)
target_pred_test_downsampled = best_model_downsampled.predict(features_test)

# Imprimir el reporte de clasificación del mejor modelo en el conjunto de prueba (downsampled)
print("\nReporte de clasificación en el conjunto de prueba (downsampled):")
print(classification_report(target_test, target_pred_test_downsampled))
print("Puntaje F1 en el conjunto de prueba (downsampled):", f1_score(target_test, target_pred_test_downsampled))
print("AUC-ROC en el conjunto de prueba (downsampled):", roc_auc_score(target_test, best_model_downsampled.predict_proba(features_test)[:, 1]))

Reporte de clasificación en el conjunto de validación (upsampled):
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      1609
           1       0.66      0.53      0.59       391

    accuracy                           0.86      2000
   macro avg       0.78      0.73      0.75      2000
weighted avg       0.85      0.86      0.85      2000

Mejores parámetros (upsampled): {'n_estimators': 180, 'max_depth': None}
Puntaje F1 en el conjunto de validación (upsampled): 0.5889046941678521
AUC-ROC en el conjunto de validación (upsampled): 0.8442583994443023

Reporte de clasificación en el conjunto de validación (downsampled):
              precision    recall  f1-score   support

           0       0.92      0.78      0.85      1609
           1       0.45      0.73      0.56       391

    accuracy                           0.78      2000
   macro avg       0.69      0.76      0.70      2000
weighted avg       0.83      0.78      0.79      2

Primero convierte las variables categóricas en variables dummy utilizando pd.get_dummies(), luego divide los datos en conjuntos de entrenamiento, validación y prueba. Luego, realiza un remuestreo de la clase minoritaria en el conjunto de entrenamiento para abordar el desequilibrio de clases. Después de entrenar varios modelos con diferentes hiperparámetros, selecciona el mejor modelo basado en el puntaje F1 en el conjunto de validación y evalúa su rendimiento en el conjunto de prueba.

### Conclusiones

<p>El modelo parece ser efectivo para predecir si un cliente abandonará pronto el banco, con una capacidad F1 de prueba (downsampled) de 0.60, lo cuál es razonable para discriminar entre clientes que abandonan y no abandonan.

Cuanto mayor sea el AUC-ROC, mejor será el rendimiento del clasificador en la clasificación binaria. Para ambos casos (upsampled y downsampled), el AUC-ROC en el conjunto de prueba es aproximadamente 0.85. </p>